<a href="https://colab.research.google.com/github/MehzHats/Boston-Data-Analysis/blob/main/Boston%20Airbnb%20EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boston Airbnb Data Analysis